In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
from datetime import datetime
import datetime
from collections import Counter

In [2]:
game_logistics_df = pd.read_csv('total_game_logistics.csv')
away_stats_df = pd.read_csv('total_away_stats.csv')
home_stats_df = pd.read_csv('total_home_stats.csv')
box_score_df = pd.read_csv('total_box_score_df.csv')
hof_df = pd.read_csv('MLB_hof.csv')
mvp_df = pd.read_csv('MLB_mvp.csv')
allstar_df = pd.read_csv('MLB_allstars.csv')
pitching_line_df = pd.read_csv('pitching_line_df.csv')
cya_df = pd.read_csv('MLB_cya.csv')
team_codes_df = pd.read_csv('team_codes.csv')
games_attended_df = pd.read_csv('sports_attendance.csv')
clemente_award_df = pd.read_csv('clemente_award.csv')
all_MLB_df = pd.read_csv('all_MLB_df.csv')
all_silver_sluggers_df = pd.read_csv('MLB_silver_sluggers.csv')
ERA_champs_df = pd.read_csv('MLB_ERA_champs.csv')
RLOY_df = pd.read_csv('MLB_RLOY.csv')
roy_df = pd.read_csv('MLB_ROY.csv')
all_gold_gloves_df = pd.read_csv('MLB_gold_gloves.csv')
batting_champs_df = pd.read_csv('MLB_batting_champs.csv')


In [40]:
home_stats_df.columns

Index(['Unnamed: 0', 'Team', 'AB', 'R', 'H', 'RBI', 'BB', 'SO', 'PA', 'BA',
       'OBP', 'SLG', 'OPS', 'Pit', 'Str', 'WPA', 'aLI', 'WPA+', 'WPA-', 'cWPA',
       'acLI', 'RE24', 'PO', 'A', 'Details', 'Date'],
      dtype='object')

In [4]:
all_player_stats_df = away_stats_df.append(home_stats_df)

all_player_stats_df.rename(columns = {'Unnamed: 0':'Name'}, inplace = True)

all_player_stats_df = all_player_stats_df[all_player_stats_df.Name != 'Team Totals']
# all_player_stats_df = all_player_stats_df[all_player_stats_df.MP != 'Did Not Play']
# all_player_stats_df = all_player_stats_df[all_player_stats_df.MP != 'Did Not Dress']

all_player_stats_df = all_player_stats_df.fillna(0)
all_player_stats_df['Games'] = 1
all_player_stats_df.sort_values(by=['OPS'], ascending=False).head(25)


,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,...,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Games
752,['Jesse BiddleP'],Atlanta Braves,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000,...,0.000,0,0,0.00,0.0,0.0,0.0,0,6/22/2018,1
11,['Jonathan SánchezP'],San Francisco Giants,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000,...,0.000,0,0,0.00,0.0,0.0,0.0,0,7/27/2007,1
346,['David Freese3B'],Los Angeles Angels of Anaheim,2.0,1.0,1.0,1.0,1.0,0.0,3.0,0.500,...,0.226,-0.016,1.98%,18.34,1.2,2.0,0.0,HR,10/2/2014,1
935,['Zach Remillard2B'],Chicago White Sox,3.0,0.0,3.0,2.0,1.0,0.0,4.0,1.000,...,0.695,0.000,0.17%,1.51,2.6,3.0,1.0,0,6/17/2023,1
351,['Chris IannettaC'],Los Angeles Angels of Anaheim,3.0,1.0,1.0,1.0,1.0,1.0,4.0,0.333,...,0.248,-0.137,1.15%,31.99,0.9,9.0,0.0,HR,10/2/2014,1
767,['Danny SantanaLF'],Atlanta Braves,3.0,1.0,2.0,0.0,1.0,0.0,4.0,0.667,...,0.144,-0.161%,-0.01%,2.34,1.0,1.0,0.0,"2B,CS",6/22/2018,1
559,['Tyler CravyP'],Milwaukee Brewers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333,...,0.000,0,0,0.00,0.0,0.0,0.0,0,9/20/2016,1
118,['Travis Ishikawa1B'],San Francisco Giants,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333,...,0.000,0,0,0.00,0.0,3.0,0.0,0,4/24/2010,1
924,['Aledmys DíazSS'],Oakland Athletics,3.0,0.0,2.0,1.0,1.0,1.0,4.0,0.667,...,0.299,-0.027,0.16%,1.40,1.1,1.0,3.0,0,3/30/2023,1
189,['Carlos BeltránRF'],St. Louis Cardinals,4.0,1.0,1.0,2.0,0.0,0.0,4.0,0.385,...,0.067,-0.038,0.53%,12.16,1.4,3.0,0.0,HR,10/14/2012,1


In [5]:
# # Points scored --> P
# # Field goals made --> FG
# # Free throws scored --> FT
# # Offensive rebounds --> OR
# # Defensive rebounds --> DR
# # Steals --> S
# # Assists --> A
# # Blocked shots --> B
# # Field goal attempts --> FGA
# # Free throws attempts --> FTA
# # Personal fouls --> PF
# # Turnovers --> T
# # games score --> GS

# # GS=P+0.4FG−0.7FGA−0.4(FTA−FT)+0.7OR+0.3DR+S+0.7A+0.7B−0.4PF−T

# # The game score metric gives a number that could be anything between 0 to 40 or even higher. If a player 
# # manages 10 on the game score, they are considered average and extraordinary at 40 or higher.


# PTS_list = all_player_stats_df["PTS"].values.tolist()
# FG_list = all_player_stats_df["FG"].values.tolist()
# FT_list = all_player_stats_df["FT"].values.tolist()
# ORB_list = all_player_stats_df["ORB"].values.tolist()
# DRB_list = all_player_stats_df["DRB"].values.tolist()
# STL_list = all_player_stats_df["STL"].values.tolist()
# AST_list = all_player_stats_df["AST"].values.tolist()
# BLK_list = all_player_stats_df["BLK"].values.tolist()
# FGA_list = all_player_stats_df["FGA"].values.tolist()
# FTA_list = all_player_stats_df["FTA"].values.tolist()
# PF_list = all_player_stats_df["PF"].values.tolist()
# TOV_list = all_player_stats_df["TOV"].values.tolist()
# GS_list = []

# for i in range(len(PTS_list)):
#     GS = ((PTS_list[i]) + (0.4*(FG_list[i])) - (0.7*(FGA_list[i])) - (0.4*((FTA_list[i]) - (FT_list[i]))) + (0.7*(ORB_list[i]))
#     + (0.3*(DRB_list[i])) + (STL_list[i]) + (0.7*(AST_list[i])) + (0.7*(BLK_list[i])) - (0.4*(PF_list[i])) - (TOV_list[i]))
    
#     GS_list.append(GS)
    
# all_player_stats_df["GS"] = GS_list    
# all_player_stats_df

In [6]:
all_player_stats_df

,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,...,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Games
0,['Hanley RamirezSS'],Florida Marlins,5.0,1.0,3.0,4.0,0.0,0.0,6.0,0.339,...,0.245,-0.071,0.03%,0.26,1.7,4.0,2.0,"HR,2B,SF",7/27/2007,1
1,['Dan Uggla2B'],Florida Marlins,6.0,1.0,1.0,0.0,0.0,0.0,6.0,0.257,...,0.039,-0.127,-0.01%,0.24,-1.0,1.0,3.0,2B,7/27/2007,1
2,['Miguel Cabrera3B'],Florida Marlins,4.0,1.0,2.0,1.0,2.0,0.0,6.0,0.332,...,0.125,-0.051,0.01%,0.20,0.9,0.0,4.0,IW,7/27/2007,1
3,['Josh WillinghamLF'],Florida Marlins,2.0,1.0,1.0,0.0,3.0,0.0,5.0,0.263,...,0.091,-0.046,0.01%,0.21,0.8,2.0,0.0,GDP,7/27/2007,1
4,['Mike Jacobs1B'],Florida Marlins,5.0,1.0,2.0,2.0,0.0,1.0,5.0,0.277,...,0.250,-0.088,0.03%,0.35,0.7,8.0,1.0,2B,7/27/2007,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,[''],San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.000,0,0,0.00,0.0,0.0,0.0,0,9/3/2023,1
988,['Seth LugoP'],San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.000,0,0,0.00,0.0,0.0,0.0,0,9/3/2023,1
989,['Nick MartinezP'],San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.000,0,0,0.00,0.0,0.0,0.0,0,9/3/2023,1
990,['Scott BarlowP'],San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0.000,0,0,0.00,0.0,0.0,0.0,0,9/3/2023,1


In [7]:
# print(all_player_stats_df[['Pit']].to_string(index=False)) 
batting_details_list = all_player_stats_df.Details.tolist()

batting_details_separated_list = []

for i in range(len(batting_details_list)):
    if batting_details_list[i] == 0:
        batting_details_separated_list.append('')
    else:
        x = batting_details_list[i].split(',')
        batting_details_separated_list.append(x)
#     for j in range(len(batting_details_list[i]))

unique_details_list = []

for i in range(len(batting_details_separated_list)):
    if len(batting_details_separated_list[i]) > 0:
        for j in range(len(batting_details_separated_list[i])):
            if '·' in batting_details_separated_list[i][j]:
                y = batting_details_separated_list[i][j].split('·')
#                 unique_details_list.append(y[1])
                
                if y[1] not in unique_details_list:
                    unique_details_list.append(batting_details_separated_list[i][j])
            elif batting_details_separated_list[i][j] not in unique_details_list:
                unique_details_list.append(batting_details_separated_list[i][j])
            else:
                continue
               
               
unique_details_list              
               

               

['HR', '2B', 'SF', 'IW', 'GDP', 'SH', 'SB', 'HBP', '3B', 'CS']

In [8]:
# batting_details_temp_list = []
batting_details_complete_list = []

print(unique_details_list)

for i in range(len(batting_details_separated_list)):
    batting_details_temp_list = []

    if len(batting_details_separated_list[i]) == 0:
        for m in range(len(unique_details_list)):
            batting_details_temp_list.append(0)
    elif len(batting_details_separated_list[i]) == 1:
        for n in range(len(unique_details_list)):
            if unique_details_list[n] == batting_details_separated_list[i][0]:
                batting_details_temp_list.append(1)
            elif '·' in batting_details_separated_list[i]:
                y = batting_details_separated_list[i].split('·')
                if y[1] == batting_details_separated_list[i]:
                    batting_details_temp_list.append((y[0]))
            else:
                batting_details_temp_list.append(0)      
    else:
        new_temp_list = []
        new_temp_duplicate_list = []
        
        for j in range(len(batting_details_separated_list[i])):
            if '·' in batting_details_separated_list[i][j]:
                y = batting_details_separated_list[i][j].split('·')
                new_temp_list.append(y[1])
                duplicate_int = int(y[0])
                new_temp_duplicate_list.append(duplicate_int)
            else:
                new_temp_list.append(batting_details_separated_list[i][j])
                new_temp_duplicate_list.append(1)
        
        for k in range(len(unique_details_list)):
            if unique_details_list[k] in new_temp_list:
                index_holder = new_temp_list.index(unique_details_list[k])
                batting_details_temp_list.append(new_temp_duplicate_list[index_holder])
                
            else:
                batting_details_temp_list.append(0)

    print(batting_details_temp_list)
    batting_details_complete_list.append(batting_details_temp_list)
                    
                    
            

['HR', '2B', 'SF', 'IW', 'GDP', 'SH', 'SB', 'HBP', '3B', 'CS']
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 

In [9]:
# new_all_player_stats_df.columns

In [10]:
# unique_details_list.index("IW")
# batting_details_complete_list
# y
# unique_details_list.index(y[])
# batting_details_separated_list[0][0]
# len(batting_details_separated_list[0])
# len(batting_details_separated_list[1])
# y[0]
# batting_details_separated_list
# new_temp_list[0]
# len(batting_details_separated_list[1])
batting_details_df = pd.DataFrame(batting_details_complete_list, columns = unique_details_list)
# batting_details_df
new_all_player_stats_df = pd.merge(all_player_stats_df, batting_details_df, right_index=True, left_index=True)

new_all_player_stats_df

,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,...,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS
0,['Hanley RamirezSS'],Florida Marlins,5.0,1.0,3.0,4.0,0.0,0.0,6.0,0.339,...,1,1,1,0,0,0,0,0,0,0
0,['Dave RobertsCF'],San Francisco Giants,4.0,1.0,2.0,0.0,1.0,1.0,5.0,0.262,...,1,1,1,0,0,0,0,0,0,0
1,['Dan Uggla2B'],Florida Marlins,6.0,1.0,1.0,0.0,0.0,0.0,6.0,0.257,...,0,1,0,0,0,0,0,0,0,0
1,['Omar VizquelSS'],San Francisco Giants,4.0,1.0,1.0,1.0,0.0,0.0,5.0,0.246,...,0,1,0,0,0,0,0,0,0,0
2,['Miguel Cabrera3B'],Florida Marlins,4.0,1.0,2.0,1.0,2.0,0.0,6.0,0.332,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,[''],San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,1,1,0,0,0,0,0,0,0,0
988,['Seth LugoP'],San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0,0,0,0,0,0,0,0,0,0
989,['Nick MartinezP'],San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0,1,1,0,0,0,0,0,0,0
990,['Scott BarlowP'],San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0,0,0,0,0,0,0,0,0,0


In [11]:
temp_name_list = new_all_player_stats_df.Name.to_list()
# temp_name_list[0]
cleaned_name_list = []

for item in temp_name_list:
    temp_name = re.findall(r"\'(.*?)'", item)
    
    if temp_name == []:
        temp_name = re.findall(r'"(.*?)"', item)
    
    cleaned_name_list.append(temp_name[0])

# temp_name = re.findall(r"\'(.*?)'", temp_name_list[0])
# cleaned_name_list

name_pos_list = []

for name in cleaned_name_list:
    for i in range(len(name)):
        j = i + 1
#         print(name[(-j)])
        if name[(-j)] == 'I':
            break
        elif (name[(-j)].isupper() == True) or (name[(-j)] == '-') or (name[(-j)].isnumeric() == True):
            continue
        else:     
            name = name[:(-i)] + '|' + name[(-i):]
            break    
    
    name_pos = name.split('|')
    name_pos_list.append(name_pos)
        
final_name_list = []
pos_list = []
        
for item in name_pos_list:
    final_name_list.append(item[0])
    try:
        pos_list.append(item[1])
    except IndexError:
        pos_list.append('')
        
        
new_all_player_stats_df['Name'] = final_name_list       
new_all_player_stats_df['Pos'] = pos_list
new_all_player_stats_df

,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,...,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,Pos
0,Hanley Ramirez,Florida Marlins,5.0,1.0,3.0,4.0,0.0,0.0,6.0,0.339,...,1,1,0,0,0,0,0,0,0,SS
0,Dave Roberts,San Francisco Giants,4.0,1.0,2.0,0.0,1.0,1.0,5.0,0.262,...,1,1,0,0,0,0,0,0,0,CF
1,Dan Uggla,Florida Marlins,6.0,1.0,1.0,0.0,0.0,0.0,6.0,0.257,...,1,0,0,0,0,0,0,0,0,2B
1,Omar Vizquel,San Francisco Giants,4.0,1.0,1.0,1.0,0.0,0.0,5.0,0.246,...,1,0,0,0,0,0,0,0,0,SS
2,Miguel Cabrera,Florida Marlins,4.0,1.0,2.0,1.0,2.0,0.0,6.0,0.332,...,0,0,1,0,0,0,0,0,0,3B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,,San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,1,0,0,0,0,0,0,0,0,
988,Seth Lugo,San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0,0,0,0,0,0,0,0,0,P
989,Nick Martinez,San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,1,1,0,0,0,0,0,0,0,P
990,Scott Barlow,San Diego Padres,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,0,0,0,0,0,0,0,0,0,P


In [12]:
# # testing = re.findall(r'(\d[A-Z])', 'Anthony Rizzo1B-2B')
# test_string = 'Corey DickersonPH-LF'
# # test_string = test_string[:-2] + '&' + test_string[-2:]
# # test_string[-2:]

# for i in range(len(test_string)):
#     j = i + 1
#     print(test_string[(-j)])
#     if (test_string[(-j)].isupper() == True) or (test_string[(-j)] == '-') or (test_string[(-j)].isnumeric() == True):
#         continue
#     else:     
#         test_string = test_string[:(-i)] + '|' + test_string[(-i):]
#         break

# len(name_pos_list)


new_all_player_stats_df.sort_values(by=['AB'], ascending=False).head(25)



,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,...,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,Pos
748,Ozzie Albies,Atlanta Braves,7.0,2.0,2.0,1.0,1.0,0.0,8.0,0.250,...,0,0,1,0,0,0,0,0,0,2B
749,Freddie Freeman,Atlanta Braves,7.0,1.0,2.0,2.0,0.0,1.0,7.0,0.331,...,0,0,0,0,0,0,0,0,0,1B
761,Dansby Swanson,Atlanta Braves,7.0,1.0,1.0,0.0,0.0,1.0,7.0,0.245,...,1,0,0,0,0,0,0,0,0,SS
761,Jonathan Schoop,Baltimore Orioles,7.0,1.0,2.0,2.0,0.0,0.0,7.0,0.212,...,1,0,0,0,0,0,0,0,0,2B
830,Logan Forsythe,Texas Rangers,6.0,1.0,4.0,3.0,0.0,1.0,6.0,0.261,...,0,0,0,0,0,0,0,0,0,3B
580,Billy Burns,Kansas City Royals,6.0,1.0,1.0,0.0,0.0,2.0,6.0,0.235,...,0,0,0,0,0,0,0,0,0,CF
750,Nick Markakis,Atlanta Braves,6.0,0.0,2.0,1.0,1.0,0.0,7.0,0.323,...,0,0,0,0,0,0,0,0,0,RF
751,Craig Gentry,Baltimore Orioles,6.0,1.0,2.0,1.0,0.0,0.0,7.0,0.246,...,0,0,0,0,0,0,0,0,0,LF-CF
751,Tyler Flowers,Atlanta Braves,6.0,0.0,0.0,0.0,0.0,3.0,6.0,0.245,...,0,0,0,0,0,0,0,0,0,C
827,Hunter Pence,Texas Rangers,6.0,0.0,2.0,3.0,0.0,0.0,6.0,0.292,...,1,0,0,0,0,1,0,0,0,LF


In [13]:
# len(cleaned_name_list)
# len(temp_name_list)
# item
# temp_name
# len(cleaned_name_list[0])
# test_string
# test_string[(-(j+1))]
new_all_player_stats_df.Games

0      1
0      1
1      1
1      1
2      1
      ..
987    1
988    1
989    1
990    1
991    1
Name: Games, Length: 1854, dtype: int64

In [14]:
col_list = new_all_player_stats_df.columns.tolist()
col_list.remove('Name')
col_list.remove('Team')
col_list.remove('BA')
col_list.remove('OBP')
col_list.remove('SLG')
col_list.remove('OPS')
col_list.remove('Pit')
col_list.remove('Str')
col_list.remove('aLI')
col_list.remove('WPA+')
col_list.remove('WPA-')
col_list.remove('cWPA')
col_list.remove('acLI')
col_list.remove('RE24')
col_list.remove('Details')
col_list.remove('Date')
# col_list.remove('Games')
# col_list.remove('Pos')

total_sum_list = []

for col in col_list: 
    sum_list = []
    if col == "Team" or col == 'Pos':           
        players_list = new_all_player_stats_df["Name"].values.tolist()
        player_pos_list = new_all_player_stats_df["Pos"].values.tolist()        
        teams_list = new_all_player_stats_df["Team"].values.tolist()
        new_teams_list = []
        new_pos_list = []
        
        for i in range(len(players_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(teams_list[i])
            new_teams_list.append(temp_list)
        
        for i in range(len(player_pos_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(player_pos_list[i])
            new_pos_list.append(temp_list)
        
        new_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
        new_teams_df = new_teams_df.drop_duplicates()
        new_teams_df = new_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
        new_teams_df = new_teams_df.set_index('Name')
        
        new_pos_df = pd.DataFrame(new_pos_list, columns = ("Name", "Pos"))
        new_pos_df = new_pos_df.drop_duplicates()
        new_pos_df = new_pos_df.groupby('Name')['Pos'].apply('/'.join).reset_index()
        new_pos_df = new_pos_df.set_index('Name')
        
    
    else:
        sum_list = new_all_player_stats_df.groupby('Name')[col].sum()
        total_sum_list.append(sum_list)
    
    
    
player_stat_sum_df = pd.DataFrame(total_sum_list)
player_stat_sum_df = player_stat_sum_df.transpose()
# player_stat_sum_df = pd.concat([temp_player_stat_sum_df, new_teams_df], axis=1, join="inner")

# player_stat_sum_df = player_stat_sum_df.rename(columns={'3P': 'Three_P', '3PA': 'Three_PA'})

H_list = player_stat_sum_df['H'].tolist()
AB_list = player_stat_sum_df['AB'].tolist()
BB_list = player_stat_sum_df['BB'].tolist()
IW_list = player_stat_sum_df['IW'].tolist()
HBP_list = player_stat_sum_df['HBP'].tolist()
SF_list = player_stat_sum_df['SF'].tolist()
double_list = player_stat_sum_df['2B'].tolist()
triple_list = player_stat_sum_df['3B'].tolist()
HR_list = player_stat_sum_df['HR'].tolist()

BA_list = []
for i in range(len(AB_list)):
    if AB_list[i] == 0:
        BA_list.append(.000)
    else:
        BA = H_list[i] / AB_list[i]
        BA_list.append(BA)

player_stat_sum_df['BA'] = BA_list
player_stat_sum_df.loc[:, "BA"] = player_stat_sum_df["BA"].map('{:.3f}'.format)


# OBP Forumla = (H + BB + HBP) / (AB + BB + HBP + SF)

OBP_list = []
for i in range(len(AB_list)):
    if float(AB_list[i] + BB_list[i] + HBP_list[i] + SF_list[i]) == 0:
        OBP_list.append(.000)
    else:
        OBP = ((H_list[i] + BB_list[i] + HBP_list[i]) / (AB_list[i] + BB_list[i] + HBP_list[i] + SF_list[i]))
        OBP_list.append(OBP)

player_stat_sum_df['OBP'] = OBP_list
player_stat_sum_df.loc[:, "OBP"] = player_stat_sum_df["OBP"].map('{:.3f}'.format)


# (1B + 2Bx2 + 3Bx3 + HRx4)/AB

SLG_list = []
for i in range(len(AB_list)):
    if AB_list[i] == 0:
        SLG_list.append(.000)
    else:
        SLG = (((H_list[i] - double_list[i] - triple_list[i] - HR_list[i]) + (2*double_list[i]) + (3*triple_list[i]) + (4*HR_list[i]))/ (AB_list[i]))
        SLG_list.append(SLG)

player_stat_sum_df['SLG'] = SLG_list
player_stat_sum_df.loc[:, "SLG"] = player_stat_sum_df["SLG"].map('{:.3f}'.format)


OPS_list = []
for i in range(len(OBP_list)):
    OPS = OBP_list[i] + SLG_list[i]
    OPS_list.append(OPS)

player_stat_sum_df['OPS'] = OPS_list
player_stat_sum_df.loc[:, "OPS"] = player_stat_sum_df["OPS"].map('{:.3f}'.format)



# Three_P_list = player_stat_sum_df['Three_P'].tolist()
# Three_PA_list = player_stat_sum_df['Three_PA'].tolist()
# Three_P_P_list = []
# for i in range(len(Three_P_list)):
#     if Three_P_list[i] == 0:
#         Three_P_P_list.append(0)
#     else:
#         Three_P_P = Three_P_list[i] / Three_PA_list[i]
#         Three_P_P_list.append(Three_P_P)

# player_stat_sum_df['3P%'] = Three_P_P_list
    
# FT_list = player_stat_sum_df['FT'].tolist()
# FTA_list = player_stat_sum_df['FTA'].tolist()
# FTP_list = []
# for i in range(len(FT_list)):
#     if FT_list[i] == 0:
#         FTP_list.append(0)
#     else:
#         FTP = FT_list[i] / FTA_list[i]
#         FTP_list.append(FTP)

# player_stat_sum_df['FT%'] = FGP_list

# player_stat_sum_df = player_stat_sum_df.rename(columns={'Three_P': '3P', 'Three_PA': '3PA'})

# player_stat_sum_df = player_stat_sum_df.fillna(0)

# player_stat_sum_df.loc[:, "FG%"] = player_stat_sum_df["FG%"].map('{:.3f}'.format)
# player_stat_sum_df.loc[:, "3P%"] = player_stat_sum_df["3P%"].map('{:.3f}'.format)
# player_stat_sum_df.loc[:, "FT%"] = player_stat_sum_df["FT%"].map('{:.3f}'.format)

# FGP_Col = player_stat_sum_df.pop("FG%")
# TPP_Col = player_stat_sum_df.pop("3P%")
# FTP_Col = player_stat_sum_df.pop("FT%")

# player_stat_sum_df.insert(6, 'FT%', FTP_Col)
# player_stat_sum_df.insert(4, '3P%', TPP_Col)
# player_stat_sum_df.insert(2, 'FG%', FGP_Col)

# first_column = player_stat_sum_df.pop('Games')
# second_column = player_stat_sum_df.pop('Team')

# player_stat_sum_df.insert(0, 'Team', second_column)
# player_stat_sum_df.insert(0, 'Games', first_column)

player_stat_sum_df = player_stat_sum_df.join(new_teams_df)
player_stat_sum_df = player_stat_sum_df.join(new_pos_df)

player_stat_sum_df.sort_values('Games', ascending = False).head(50)


,AB,R,H,RBI,BB,SO,PA,WPA,PO,A,...,SB,HBP,3B,CS,BA,OBP,SLG,OPS,Team,Pos
Name,,,,,,,,,,,,,,,,,,,,,
,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,1.0,1.0,0.0,0.0,0.000,1.000,0.000,1.000,Minnesota Twins/Los Angeles Angels of Anaheim/...,
Yadier Molina,67.0,5.0,17.0,9.0,4.0,11.0,75.0,0.161,152.0,6.0,...,1.0,1.0,0.0,0.0,0.254,0.306,0.299,0.604,St. Louis Cardinals,C/C-1B
Matt Carpenter,52.0,5.0,9.0,4.0,3.0,20.0,58.0,-0.212,44.0,19.0,...,0.0,0.0,0.0,0.0,0.173,0.218,0.250,0.468,St. Louis Cardinals/San Diego Padres,PH/2B/3B/1B
Tommy Pham,31.0,4.0,2.0,0.0,8.0,12.0,39.0,-0.241,16.0,0.0,...,1.0,1.0,0.0,1.0,0.065,0.275,0.065,0.340,St. Louis Cardinals,CF-LF/CF/PH/LF
Jedd Gyorko,34.0,4.0,8.0,1.0,3.0,7.0,37.0,-0.386,7.0,20.0,...,0.0,0.0,0.0,1.0,0.235,0.297,0.265,0.562,San Diego Padres/St. Louis Cardinals,2B/SS/3B
David Freese,23.0,4.0,6.0,7.0,5.0,7.0,28.0,0.463,7.0,8.0,...,0.0,0.0,0.0,0.0,0.261,0.393,0.435,0.828,St. Louis Cardinals/Los Angeles Angels of Anah...,3B/DH/1B/PH
Adrian Gonzalez,29.0,4.0,7.0,8.0,1.0,6.0,31.0,-0.117,76.0,6.0,...,1.0,0.0,2.0,0.0,0.241,0.267,0.621,0.887,Los Angeles Dodgers,1B/PH-1B
Stephen Piscotty,32.0,6.0,15.0,9.0,4.0,3.0,36.0,0.367,16.0,0.0,...,0.0,0.0,0.0,0.0,0.469,0.528,0.469,0.997,St. Louis Cardinals/Oakland Athletics,LF-RF/LF/CF/RF
Greg Garcia,11.0,0.0,1.0,0.0,3.0,2.0,14.0,-0.126,1.0,2.0,...,0.0,0.0,0.0,0.0,0.091,0.267,0.364,0.630,St. Louis Cardinals,2B/PH/3B/PH-2B


In [15]:
# player_stat_sum_df.reset_index(inplace=True)
new_all_player_stats_df.columns

Index(['Name', 'Team', 'AB', 'R', 'H', 'RBI', 'BB', 'SO', 'PA', 'BA', 'OBP',
       'SLG', 'OPS', 'Pit', 'Str', 'WPA', 'aLI', 'WPA+', 'WPA-', 'cWPA',
       'acLI', 'RE24', 'PO', 'A', 'Details', 'Date', 'Games', 'HR', '2B', 'SF',
       'IW', 'GDP', 'SH', 'SB', 'HBP', '3B', 'CS', 'Pos'],
      dtype='object')

In [16]:
col_list = new_all_player_stats_df.columns.tolist()
col_list.remove('Name')
col_list.remove('BA')
col_list.remove('SLG')
col_list.remove('OPS')
col_list.remove('Pos')
col_list.remove('OBP')
col_list.remove('Pit')
col_list.remove('Str')
col_list.remove('WPA')
col_list.remove('aLI')
col_list.remove('WPA+')
col_list.remove('WPA-')
col_list.remove('cWPA')
col_list.remove('Details')
col_list.remove('Date')
col_list.remove('acLI')
col_list.remove('RE24')

total_average_list = []

for col in col_list: 
    average_list = []
    teams_list = []
    
    if col == "Games":
        average_list = new_all_player_stats_df.groupby('Name')[col].sum()
        total_average_list.append(average_list)
        
    elif col == "Team":           
        players_list = new_all_player_stats_df["Name"].values.tolist()
        teams_list = new_all_player_stats_df["Team"].values.tolist()
        new_teams_list = []
        
        for i in range(len(players_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(teams_list[i])
            new_teams_list.append(temp_list)
        
        new_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
        new_teams_df = new_teams_df.drop_duplicates()

        new_teams_df = new_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
        new_teams_df = new_teams_df.set_index('Name')

    else:
        average_list = new_all_player_stats_df.groupby('Name')[col].mean()
        total_average_list.append(average_list)
    
temp_player_stat_average_df = pd.DataFrame(total_average_list)
temp_player_stat_average_df = temp_player_stat_average_df.transpose()
player_stat_average_df = pd.concat([temp_player_stat_average_df, new_teams_df], axis=1, join="inner")

# player_stat_average_df = player_stat_average_df.rename(columns={'3P': 'Three_P', '3PA': 'Three_PA'})

# FG_list2 = player_stat_average_df['FG'].tolist()
# FGA_list2 = player_stat_average_df['FGA'].tolist()
# FGP_list2 = []
# for i in range(len(FG_list2)):
#     if FGA_list2[i] == 0:
#         FGP_list2.append(0)
#     else:
#         FGP2 = FG_list2[i] / FGA_list2[i]
#         FGP_list2.append(FGP2)

# player_stat_average_df['FG%'] = FGP_list2
        
# Three_P_list2 = player_stat_average_df['Three_P'].tolist()
# Three_PA_list2 = player_stat_average_df['Three_PA'].tolist()
# Three_P_P_list2 = []
# for i in range(len(Three_P_list2)):
#     if Three_P_list2[i] == 0:
#         Three_P_P_list2.append(0)
#     else:
#         Three_P_P2 = Three_P_list2[i] / Three_PA_list2[i]
#         Three_P_P_list2.append(Three_P_P2)

# player_stat_average_df['3P%'] = Three_P_P_list2
    
# FT_list2 = player_stat_average_df['FT'].tolist()
# FTA_list2 = player_stat_average_df['FTA'].tolist()
# FTP_list2 = []
# for i in range(len(FT_list2)):
#     if FT_list2[i] == 0:
#         FTP_list2.append(0)
#     else:
#         FTP2 = FT_list2[i] / FTA_list2[i]
#         FTP_list2.append(FTP2)

# player_stat_average_df['FT%'] = FGP_list2
      
# player_stat_average_df = player_stat_average_df.rename(columns={'Three_P': '3P', 'Three_PA': '3PA'})

# player_stat_average_df = player_stat_average_df.fillna(0)

# player_stat_average_df.loc[:, "FG%"] = player_stat_average_df["FG%"].map('{:.3f}'.format)
# player_stat_average_df.loc[:, "3P%"] = player_stat_average_df["3P%"].map('{:.3f}'.format)
# player_stat_average_df.loc[:, "FT%"] = player_stat_average_df["FT%"].map('{:.3f}'.format)

# FGP_Col = player_stat_average_df.pop("FG%")
# TPP_Col = player_stat_average_df.pop("3P%")
# FTP_Col = player_stat_average_df.pop("FT%")

# player_stat_average_df.insert(6, 'FT%', FTP_Col)
# player_stat_average_df.insert(4, '3P%', TPP_Col)
# player_stat_average_df.insert(2, 'FG%', FGP_Col)

player_stat_average_df = player_stat_average_df.round(2)

first_column2 = player_stat_average_df.pop('Games')
second_column2 = player_stat_average_df.pop('Team')

player_stat_average_df.insert(0, 'Team', second_column2)
player_stat_average_df.insert(0, 'Games', first_column2)

player_stat_average_df

,Games,Team,AB,R,H,RBI,BB,SO,PA,PO,...,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS
Name,,,,,,,,,,,,,,,,,,,,,
,44.0,Minnesota Twins/Los Angeles Angels of Anaheim/...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.09,0.2,0.0,0.02,0.09,0.02,0.02,0.02,0.0,0.0
A.J. Burnett,1.0,Philadelphia Phillies,3.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0
A.J. Ellis,5.0,Los Angeles Dodgers,3.2,0.4,0.4,0.2,0.2,0.6,3.6,8.2,...,0.20,0.2,0.0,0.00,0.00,0.20,0.20,0.20,0.0,0.0
A.J. Minter,1.0,Atlanta Braves,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0
A.J. Pierzynski,1.0,Atlanta Braves,3.0,1.0,1.0,0.0,0.0,0.0,3.0,8.0,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zack Greinke,1.0,Arizona Diamondbacks,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0
Zack Littell,1.0,San Francisco Giants,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,1.00,0.0,0.0
Zack Short,1.0,Detroit Tigers,3.0,1.0,1.0,3.0,0.0,2.0,4.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0


In [17]:
player_stat_average_df.sort_values(by=['Games'], ascending=False).head(40)

,Games,Team,AB,R,H,RBI,BB,SO,PA,PO,...,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS
Name,,,,,,,,,,,,,,,,,,,,,
,44.0,Minnesota Twins/Los Angeles Angels of Anaheim/...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.09,0.20,0.00,0.02,0.09,0.02,0.02,0.02,0.00,0.00
Yadier Molina,19.0,St. Louis Cardinals,3.53,0.26,0.89,0.47,0.21,0.58,3.95,8.00,...,0.05,0.00,0.00,0.00,0.05,0.00,0.05,0.05,0.00,0.00
Matt Carpenter,15.0,St. Louis Cardinals/San Diego Padres,3.47,0.33,0.60,0.27,0.20,1.33,3.87,2.93,...,0.07,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Tommy Pham,11.0,St. Louis Cardinals,2.82,0.36,0.18,0.00,0.73,1.09,3.55,1.45,...,0.00,0.00,0.00,0.00,0.27,0.00,0.09,0.09,0.00,0.09
Jedd Gyorko,10.0,San Diego Padres/St. Louis Cardinals,3.40,0.40,0.80,0.10,0.30,0.70,3.70,0.70,...,0.00,0.10,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.10
David Freese,9.0,St. Louis Cardinals/Los Angeles Angels of Anah...,2.56,0.44,0.67,0.78,0.56,0.78,3.11,0.78,...,0.11,0.11,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adrian Gonzalez,9.0,Los Angeles Dodgers,3.22,0.44,0.78,0.89,0.11,0.67,3.44,8.44,...,0.22,0.11,0.00,0.11,0.00,0.00,0.11,0.00,0.22,0.00
Stephen Piscotty,9.0,St. Louis Cardinals/Oakland Athletics,3.56,0.67,1.67,1.00,0.44,0.33,4.00,1.78,...,0.00,0.00,0.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00
Greg Garcia,8.0,St. Louis Cardinals,1.38,0.00,0.12,0.00,0.38,0.25,1.75,0.12,...,0.12,0.00,0.12,0.25,0.00,0.00,0.00,0.00,0.00,0.00


In [18]:
# val = all_player_stats_df['MP'].values[0]
# val2 = all_player_stats_df['MP'].values[1]

# datetime_object = datetime.datetime.strptime(val, '%M:%S').time()
# datetime_object2 = datetime.datetime.strptime(val2, '%M:%S').time()

# timeList = [val, val2]

# mysum = datetime.timedelta()
# for i in timeList:
#     (m, s) = i.split(':')
#     d = datetime.timedelta(minutes=int(m), seconds=int(s))
#     mysum += d
# print(str(mysum))

# time_list = all_player_stats_df['MP']
# time_sum_list = []
# for i in range(len(time_list)):
#     val = all_player_stats_df['MP'].values[i]
#     datetime_object = datetime.datetime.strptime(val, '%M:%S').time()


In [19]:
# box_score_df['Details'][3]

In [20]:
box_score_df = box_score_df.tail(-1)
box_score_df = box_score_df.rename(columns={"Unnamed: 0": "Date", "Unnamed: 2": "Details"})
box_score_df = box_score_df.drop(columns=['Home_Away'])

temp_box_score_df = box_score_df.dropna(subset = ['R'])

index_list = temp_box_score_df.index.tolist()
home_away_list = []

for i in range(len(index_list)):
    if (i % 2) == 0:
        home_away_list.append('Away')
    else:
        home_away_list.append('Home')

temp_box_score_df.insert(1, 'Home_Away', home_away_list)


temp_box_score_df.tail(60)

,Date,Home_Away,Details,1,10,11,12,13,14,15,...,4,5,6,7,8,9,E,H,R,Team
102,"('7/26/2016',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0,0.0,7.0,2.0,Atlanta Braves
103,"('7/26/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0,1.0,5.0,0.0,Minnesota Twins
105,"('7/29/2016',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1,0.0,8.0,1.0,Seattle Mariners
106,"('7/29/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,6.0,1.0,0.0,X,0.0,14.0,12.0,Chicago Cubs
108,"('8/7/2016',)",Away,via Sports Logos.netAbout logos,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0,1.0,11.0,6.0,Atlanta Braves
109,"('8/7/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,2.0,0,0.0,9.0,3.0,St. Louis Cardinals
111,"('9/3/2016',)",Away,via Sports Logos.netAbout logos,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0,1.0,5.0,1.0,St. Louis Cardinals
112,"('9/3/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,2.0,5.0,X,1.0,12.0,9.0,Cincinnati Reds
114,"('9/20/2016',)",Away,via Sports Logos.netAbout logos,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,1.0,0.0,0.0,0,2.0,13.0,6.0,Pittsburgh Pirates
115,"('9/20/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,0.0,0.0,0,0.0,7.0,3.0,Milwaukee Brewers


In [21]:
# final_score_list = box_score_df['R'].tolist()
# len(final_score_list)
temp_box_score_df

,Date,Home_Away,Details,1,10,11,12,13,14,15,...,4,5,6,7,8,9,E,H,R,Team
1,"('7/27/2007',)",Away,via Sports Logos.netAbout logos,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,1.0,0.0,2,2.0,15.0,10.0,Florida Marlins
2,"('7/27/2007',)",Home,via Sports Logos.netAbout logos,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,4.0,6.0,0.0,0.0,X,0.0,14.0,12.0,San Francisco Giants
4,"('7/28/2007',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0,2.0,6.0,3.0,Florida Marlins
5,"('7/28/2007',)",Home,via Sports Logos.netAbout logos,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,2,0.0,9.0,4.0,San Francisco Giants
8,"('9/5/2008',)",Away,via Sports Logos.netAbout logos,0.0,0.0,3.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1,0.0,9.0,4.0,Florida Marlins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,"('6/17/2023',)",Home,via Sports Logos.netAbout logos,2.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0,2.0,8.0,3.0,Seattle Mariners
189,"('7/15/2023',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,2.0,1.0,0,1.0,6.0,4.0,Boston Red Sox
190,"('7/15/2023',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0.0,0.0,2.0,0.0,X,0.0,12.0,10.0,Chicago Cubs
192,"('9/3/2023',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0,2.0,4.0,0.0,San Francisco Giants


In [22]:
date_list = temp_box_score_df['Date'].tolist()
final_score_list = temp_box_score_df['R'].tolist()
win_loss_list = []
home_team_wins = 0
away_team_wins = 0
home_team_losses = 0
away_team_losses = 0
j = 1

try:
    for i in range(len(final_score_list)):
#         print(final_score_list[i])
#         print(date_list[i])
        if (i + 2) != len(final_score_list) and date_list[i] == date_list[i+2]:
            date_list[i] = str(date_list[i]) + 'Game 1'
            date_list[i + 1] = str(date_list[i + 1]) + 'Game 1'
            date_list[i + 2] = str(date_list[i + 2]) + 'Game 2'
            date_list[i + 3] = str(date_list[i + 3]) + 'Game 2'
        if date_list[i] == date_list[i+1]:
            print(f'Game {j}: {date_list[i]}')
            if final_score_list[i] > final_score_list[i + 1]:
                win_loss_list.append('W')
                win_loss_list.append('L')
                away_team_wins = away_team_wins + 1
                home_team_losses = home_team_losses + 1
                print('Away Team Won')
                j = j + 1
            else:
                win_loss_list.append('L')
                win_loss_list.append('W')
                home_team_wins = home_team_wins + 1
                away_team_losses = away_team_losses + 1
                print('Home Team Won')
                j = j + 1
        else:
            continue
except IndexError:
    print("An exception was made")

home_win_loss_percentage = (home_team_wins/(home_team_wins + home_team_losses))
away_win_loss_percentage = (away_team_wins/(away_team_wins + away_team_losses))

Game 1: ('7/27/2007',)
Home Team Won
Game 2: ('7/28/2007',)
Home Team Won
Game 3: ('9/5/2008',)
Away Team Won
Game 4: ('9/6/2008',)
Home Team Won
Game 5: ('7/25/2009',)
Home Team Won
Game 6: ('7/26/2009',)
Away Team Won
Game 7: ('8/22/2009',)
Away Team Won
Game 8: ('4/24/2010',)
Home Team Won
Game 9: ('9/26/2010',)
Home Team Won
Game 10: ('10/28/2010',)
Home Team Won
Game 11: ('7/14/2012',)
Home Team Won
Game 12: ('9/11/2012',)
Home Team Won
Game 13: ('10/14/2012',)
Away Team Won
Game 14: ('5/24/2013',)
Away Team Won
Game 15: ('8/21/2013',)
Away Team Won
Game 16: ('8/30/2013',)
Home Team Won
Game 17: ('6/26/2014',)
Home Team Won
Game 18: ('7/5/2014',)
Home Team Won
Game 19: ('7/11/2014',)
Home Team Won
Game 20: ('7/20/2014',)
Home Team Won
Game 21: ('8/7/2014',)
Away Team Won
Game 22: ('8/31/2014',)
Home Team Won
Game 23: ('10/2/2014',)
Away Team Won
Game 24: ('5/23/2015',)
Home Team Won
Game 25: ('6/6/2015',)
Home Team Won
Game 26: ('6/19/2015',)
Away Team Won
Game 27: ('8/23/2015',)


In [23]:
i

123

In [24]:
home_team_record = ['Home',home_team_wins,home_team_losses,home_win_loss_percentage]
away_team_record = ['Away',away_team_wins,away_team_losses,away_win_loss_percentage]

home_away_record_df = pd.DataFrame(columns = ['', 'Wins', 'Losses','Win%'])
  
home_away_record_df.loc[len(home_away_record_df)] = home_team_record
home_away_record_df.loc[len(home_away_record_df)] = away_team_record
home_away_record_df = home_away_record_df.set_index('')
home_away_record_df.loc[:, "Win%"] = home_away_record_df["Win%"].map('{:.3f}'.format)

home_away_record_df

,Wins,Losses,Win%
,,,
Home,39,23,0.629
Away,23,39,0.371


In [25]:
temp_box_score_df['Decision'] = win_loss_list
temp_box_score_df

/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_73388/2583124398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_box_score_df['Decision'] = win_loss_list


,Date,Home_Away,Details,1,10,11,12,13,14,15,...,5,6,7,8,9,E,H,R,Team,Decision
1,"('7/27/2007',)",Away,via Sports Logos.netAbout logos,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,0.0,2,2.0,15.0,10.0,Florida Marlins,L
2,"('7/27/2007',)",Home,via Sports Logos.netAbout logos,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,6.0,0.0,0.0,X,0.0,14.0,12.0,San Francisco Giants,W
4,"('7/28/2007',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0,2.0,6.0,3.0,Florida Marlins,L
5,"('7/28/2007',)",Home,via Sports Logos.netAbout logos,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,2,0.0,9.0,4.0,San Francisco Giants,W
8,"('9/5/2008',)",Away,via Sports Logos.netAbout logos,0.0,0.0,3.0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1,0.0,9.0,4.0,Florida Marlins,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,"('6/17/2023',)",Home,via Sports Logos.netAbout logos,2.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,0.0,0,2.0,8.0,3.0,Seattle Mariners,L
189,"('7/15/2023',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,2.0,1.0,0,1.0,6.0,4.0,Boston Red Sox,L
190,"('7/15/2023',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,2.0,0.0,X,0.0,12.0,10.0,Chicago Cubs,W
192,"('9/3/2023',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0,2.0,4.0,0.0,San Francisco Giants,L


In [26]:
team_records_df = pd.DataFrame(columns = ['Team', 'Games', 'Wins', 'Losses','Win%'])

for item in temp_box_score_df.Team.unique():
    team_wins = len(temp_box_score_df.loc[(temp_box_score_df['Team'] == item) & (temp_box_score_df['Decision'] == 'W')].index)
    team_losses = len(temp_box_score_df.loc[(temp_box_score_df['Team'] == item) & (temp_box_score_df['Decision'] == 'L')].index)
    
    team_win_percentage = (team_wins/(team_wins + team_losses))
    team_total_games = (team_wins + team_losses)
    
    team_record_list = [item,team_total_games,team_wins,team_losses,team_win_percentage]
    team_records_df.loc[len(team_records_df)] = team_record_list

team_records_df = team_records_df.set_index('Team')
team_records_df.loc[:, "Win%"] = team_records_df["Win%"].map('{:.3f}'.format)

team_records_df.sort_values(by=['Games','Win%'], ascending=False)

,Games,Wins,Losses,Win%
Team,,,,
St. Louis Cardinals,21,9,12,0.429
San Francisco Giants,11,7,4,0.636
Los Angeles Dodgers,11,5,6,0.455
Chicago Cubs,6,5,1,0.833
San Diego Padres,6,2,4,0.333
Florida Marlins,5,2,3,0.400
Oakland Athletics,5,2,3,0.400
Chicago White Sox,4,3,1,0.750
Los Angeles Angels of Anaheim,4,2,2,0.500


In [27]:
player_records_df = pd.DataFrame(columns = ['Name', 'Games', 'Wins', 'Losses','Win%'])

temp_date_list = temp_box_score_df['Date'].to_list()
cleaned_dates_list = []

temp_box_score_df

for item in temp_date_list:
    cleaned_date = re.findall(r"'([^']*)'", item)
    cleaned_dates_list.append(cleaned_date[0])


temp_box_score_df.drop('Date', axis = 1, inplace = True)
temp_box_score_df['Date'] = cleaned_dates_list

for player in new_all_player_stats_df.Name.unique():
    
    temp_df = new_all_player_stats_df.loc[(new_all_player_stats_df['Name'] == player)]
    temp_merge_df = pd.merge(temp_df, temp_box_score_df, on=['Team','Date'])
         
    player_wins = 0
    player_losses = 0
        
    for item in temp_merge_df['Decision']:
        if item == 'L':
            player_losses = player_losses + 1
        else:
            player_wins = player_wins + 1

        player_win_percentage = (player_wins/(player_wins + player_losses))
        player_total_games = (player_wins + player_losses)

        player_record_list = [player,player_total_games,player_wins,player_losses,player_win_percentage]
        player_records_df.loc[len(player_records_df)] = player_record_list

player_records_df.loc[:, "Win%"] = player_records_df["Win%"].map('{:.3f}'.format)
player_records_df = player_records_df.sort_values(by=['Games'], ascending=False)                                      

player_records_df = player_records_df.drop_duplicates(subset=['Name'])
player_records_df = player_records_df.set_index('Name')

player_records_df = player_records_df.sort_values(by=['Games','Win%'], ascending=False)                                      

player_records_df


/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_73388/2114474786.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_box_score_df['Date'] = cleaned_dates_list


,Games,Wins,Losses,Win%
Name,,,,
,44,22,22,0.500
Yadier Molina,19,9,10,0.474
Matt Carpenter,15,7,8,0.467
Tommy Pham,11,5,6,0.455
Jedd Gyorko,10,2,8,0.200
...,...,...,...,...
Cory Rasmus,1,0,1,0.000
Josh Reddick,1,0,1,0.000
Sam Fuld,1,0,1,0.000


In [28]:
player_stat_average_df.reset_index(inplace=True)
hof_seen_df = pd.merge(player_stat_average_df, hof_df, on="Name")
hof_seen_df.drop('Unnamed: 0', axis = 1, inplace = True)
hof_seen_df.rename(columns = {'Unnamed: 2': 'Life'}, inplace = True)

hof_seen_df

,Name,Games,Team,AB,R,H,RBI,BB,SO,PA,...,SH,SB,HBP,3B,CS,Life,Voted By,Inducted As,Votes,% of Ballots
0,Billy Hamilton,2.0,Cincinnati Reds,4.0,1.5,1.0,0.0,0.5,1.0,4.5,...,0.0,0.0,0.0,0.0,0.0,1866-1940,Veterans,Player,NaN,NaN
1,David Ortiz,1.0,Boston Red Sox,3.0,0.0,0.0,0.0,0.0,2.0,3.0,...,0.0,0.0,0.0,0.0,0.0,1975-Living,BBWAA,Player,307.0,77.9%
2,Derek Jeter,1.0,New York Yankees,5.0,0.0,1.0,1.0,0.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,1974-Living,BBWAA,Player,396.0,99.7%
3,Tony Gwynn,2.0,San Diego Padres/Philadelphia Phillies,2.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1960-2014,BBWAA,Player,532.0,97.6%


In [29]:
player_stat_average_df.reset_index(inplace=True)
player_stat_average_df.rename(columns={'Name': 'Player'}, inplace=True)
mvp_seen_df = pd.merge(player_stat_average_df, mvp_df, on="Player")
mvp_seen_df.drop('Unnamed: 0', axis = 1, inplace = True)


mvp_seen_df

,index,Player,Games,Team,AB,R,H,RBI,BB,SO,...,SF,IW,GDP,SH,SB,HBP,3B,CS,Year,League
0,7,Aaron Judge,2.0,New York Yankees,4.00,0.00,1.00,0.00,0.00,2.00,...,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,2022,AL
1,36,Albert Pujols,7.0,St. Louis Cardinals/Los Angeles Angels of Anaheim,4.00,0.57,0.71,0.43,0.43,0.57,...,0.00,0.00,0.0,0.0,0.0,0.14,0.0,0.00,2009,NL
2,36,Albert Pujols,7.0,St. Louis Cardinals/Los Angeles Angels of Anaheim,4.00,0.57,0.71,0.43,0.43,0.57,...,0.00,0.00,0.0,0.0,0.0,0.14,0.0,0.00,2008,NL
3,36,Albert Pujols,7.0,St. Louis Cardinals/Los Angeles Angels of Anaheim,4.00,0.57,0.71,0.43,0.43,0.57,...,0.00,0.00,0.0,0.0,0.0,0.14,0.0,0.00,2005,NL
4,58,Andrew McCutchen,2.0,Pittsburgh Pirates,4.50,0.00,1.50,1.50,0.00,1.00,...,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,2013,NL
5,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,...,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,2004,NL
6,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,...,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,2003,NL
7,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,...,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,2002,NL
8,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,...,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,2001,NL
9,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,...,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00,1993,NL


In [30]:
# all_league_team_list = all_MLB_df['Team'].to_list()
# # all_league_team_list = all_league_df['Team'].to_list()
# all_MLB_df.drop('Team', axis = 1, inplace = True)
# all_MLB_df['All_MLB_Team'] = all_league_team_list

# all_league_player_list = all_MLB_df['Name'].to_list()
# cleaned_all_league_player_list = []

# for player in all_league_player_list:
#     cap_letter_check = player[-1]
#     if cap_letter_check.isupper() == True:
#         cleaned_player = player[:-1]
#         cleaned_all_league_player_list.append(cleaned_player)
#     else:
#         cleaned_all_league_player_list.append(player)

# consolidated_all_league_dict = dict(Counter(cleaned_all_league_player_list))
# aggregate_all_league_df = pd.DataFrame(list(consolidated_all_league_dict.items()), columns = ['Player', 'X_All_NBA'])

    
# # # Drop that column
# # all_league_df.drop('Player', axis = 1, inplace = True)

# # # Put whatever series you want in its place
# # all_league_df['Player'] = cleaned_all_league_player_list
# # all_league_df.drop('Unnamed: 0', axis = 1, inplace = True)

# player_stat_average_df.reset_index(inplace=True)
# player_stat_average_df.rename(columns={'Name': 'Player'}, inplace=True)
# aggregate_all_league_df = pd.merge(player_stat_average_df, aggregate_all_league_df, on="Player")

# aggregate_all_league_df.drop('index', axis = 1, inplace = True)
# aggregate_all_league_df.drop('level_0', axis = 1, inplace = True)

# aggregate_all_league_df.sort_values(by=['X_All_NBA'], ascending=False)

In [31]:
# ## End goal is to have a list that reads [1st: X, 2nd: Y, 3rd: Z] to show for each player

# # all_league_player_list
# # all_league_team_list

# all_league_df['Player'] = cleaned_all_league_player_list

# summary_list = []

# for player in cleaned_all_league_player_list:
#     player_all_nba_team_list = all_league_df.loc[all_league_df['Player'] == player]['All_NBA_Team'].to_list()
    
#     for i in range(len(player_all_nba_team_list)):
#         if i == 0:
#             first_team_counter = 0
#             second_team_counter = 0
#             three_team_counter = 0

#         if player_all_nba_team_list[i] == '1st':
#             first_team_counter = first_team_counter + 1
#         elif player_all_nba_team_list[i] == '2nd':
#             second_team_counter = second_team_counter + 1
#         else:
#             three_team_counter = three_team_counter + 1

#     summary = [f'1st Team: {first_team_counter}',f'2nd Team: {second_team_counter}',f'3rd Team: {three_team_counter}']
#     summary_list.append(summary)  
        
# all_league_df['All_League_List'] = summary_list
# all_league_list_df = all_league_df.drop(['Unnamed: 0','Year','All_NBA_Team'], axis=1)

# # player_stat_average_df.reset_index(inplace=True)
# # player_stat_average_df.rename(columns={'Name': 'Player'}, inplace=True)
# all_league_list_df = pd.merge(aggregate_all_league_df, all_league_list_df, on="Player")

# all_league_list_df.drop_duplicates(subset = 'Player', inplace=True)
# # all_league_list_df.sort_values(by=['X_All_NBA'], ascending=False)

# all_league_simple_df = all_league_list_df[['Player', 'Games','X_All_NBA','All_League_List']].copy()
# all_league_simple_df.sort_values(by=['X_All_NBA'], ascending=False)

In [32]:
allstar_df

,Unnamed: 0,Lineup Spot,Player,Position,Year
0,0,1.0,Pepper Martin,3B,1933: Game 1
1,1,2.0,Frankie Frisch,2B,1933: Game 1
2,2,3.0,Chuck Klein,RF,1933: Game 1
3,3,4.0,Chick Hafey,LF,1933: Game 1
4,4,5.0,Bill Terry,1B,1933: Game 1
...,...,...,...,...,...
6112,6112,NaN,Pablo López,P,2023: Game 1
6113,6113,NaN,Michael Lorenzen,P,2023: Game 1
6114,6114,NaN,Shane McClanahan,P,2023: Game 1
6115,6115,NaN,Jordan Romano,P,2023: Game 1


In [33]:
all_star_list = allstar_df['Player'].to_list()

i = 0
allstars_seen_list = []
allstar_appearance_counter_list = []

for player in new_all_player_stats_df.Name.unique():
    allstar_appearance_counter = 0
    
    for i in range(len(all_star_list)):
        if (player in all_star_list[i]) == True:
            if player not in allstars_seen_list:
                allstars_seen_list.append(player)
                allstar_appearance_counter = allstar_appearance_counter + 1
            else:
                allstar_appearance_counter = allstar_appearance_counter + 1
    
    allstar_appearance_counter_list.append(allstar_appearance_counter)
    i = i + 1            
    
allstars_seen_counter_list = [i for i in allstar_appearance_counter_list if i != 0]
    
allstars_seen_df = pd.DataFrame(columns = ['Player', 'Appearances'])
allstars_seen_df['Player'] = allstars_seen_list
allstars_seen_df['Appearances'] = allstars_seen_counter_list
allstars_seen_df.sort_values(by=['Appearances'], ascending=False)

,Player,Appearances
35,,6117
46,Tony Gwynn,15
140,Derek Jeter,14
6,Barry Bonds,14
4,Miguel Cabrera,12
...,...,...
214,Adam Duvall,1
215,Mike Foltynewicz,1
216,Michael Lorenzen,1
217,Orlando Arcia,1


In [34]:
# all_gold_gloves_seen_df = pd.merge(player_stat_average_df, all_gold_gloves_df, on="Player")

# all_gold_gloves_seen_df.drop('level_0', axis = 1, inplace = True)
# all_gold_gloves_seen_df.drop('index', axis = 1, inplace = True)
# all_gold_gloves_seen_df.drop('Unnamed: 0', axis = 1, inplace = True)

# # all_gold_gloves_seen_df.sort_values(by=['X_DPOY'], ascending=False)
# all_gold_gloves_seen_df

## 

In [35]:
home_teams_seen_list = games_attended_df['Home'].unique()
away_teams_seen_list = games_attended_df['Visitor'].unique()
all_teams_seen_list = list(home_teams_seen_list)

# for away_team in away_teams_seen_list:
#     all_teams_seen_list.append(away_team)

[all_teams_seen_list.append(x) for x in away_teams_seen_list if x not in all_teams_seen_list]

    
mlb_teams_list = [x for x in team_codes_df['MLB_Teams'].to_list() if str(x) != 'nan']
nba_teams_list = [x for x in team_codes_df['NBA_Teams'].to_list() if str(x) != 'nan']
nfl_teams_list = [x for x in team_codes_df['NFL_Teams'].to_list() if str(x) != 'nan']
nhl_teams_list = [x for x in team_codes_df['NHL_Teams'].to_list() if str(x) != 'nan']
mls_teams_list = [x for x in team_codes_df['MLS_Codes'].to_list() if str(x) != 'nan']

home_mlb_teams_seen_list = []
home_nba_teams_seen_list = []
home_nfl_teams_seen_list = []
home_nhl_teams_seen_list = []
home_mls_teams_seen_list = []

for team in home_teams_seen_list:
    if team in mlb_teams_list:
        home_mlb_teams_seen_list.append(team)
    elif team in nba_teams_list:
        home_nba_teams_seen_list.append(team)
    elif team in nfl_teams_list:
        home_nfl_teams_seen_list.append(team)
    elif team in nhl_teams_list:
        home_nhl_teams_seen_list.append(team)
    else: 
        team = team.replace(' ', '-')
        for mls_team in mls_teams_list:
            if team in mls_team:
                home_mls_teams_seen_list.append(team)
            else:
                continue

                
all_mlb_teams_seen_list = []
all_nba_teams_seen_list = []
all_nfl_teams_seen_list = []
all_nhl_teams_seen_list = []
all_mls_teams_seen_list = []

for team in all_teams_seen_list:
    if team in mlb_teams_list:
        all_mlb_teams_seen_list.append(team)
    elif team in nba_teams_list:
        all_nba_teams_seen_list.append(team)
    elif team in nfl_teams_list:
        all_nfl_teams_seen_list.append(team)
    elif team in nhl_teams_list:
        all_nhl_teams_seen_list.append(team)
    else: 
        team = team.replace(' ', '-')
        for mls_team in mls_teams_list:
            if team in mls_team:
                all_mls_teams_seen_list.append(team)
            else:
                continue
                
mlb_stadiums_visited = (len(home_mlb_teams_seen_list)/((len(mlb_teams_list))-1))
nba_stadiums_visited = (len(home_nba_teams_seen_list)/len(nba_teams_list))
nfl_stadiums_visited = (len(home_nfl_teams_seen_list)/len(nfl_teams_list))
nhl_stadiums_visited = (len(home_nhl_teams_seen_list)/len(nhl_teams_list))
mls_stadiums_visited = (len(home_mls_teams_seen_list)/len(mls_teams_list))

mlb_teams_seen = (len(all_mlb_teams_seen_list)/len(mlb_teams_list))
nba_teams_seen = (len(all_nba_teams_seen_list)/len(nba_teams_list))
nfl_teams_seen = (len(all_nfl_teams_seen_list)/len(nfl_teams_list))
nhl_teams_seen = (len(all_nhl_teams_seen_list)/len(nhl_teams_list))
mls_teams_seen = (len(all_mls_teams_seen_list)/len(mls_teams_list))

games_dict = {}

games_dict['League'] = ['MLB','NBA','NFL','NHL','MLS']
games_dict['League_Teams_Total'] = [((len(mlb_teams_list))-1),len(nba_teams_list),len(nfl_teams_list),len(nhl_teams_list),len(mls_teams_list)]
games_dict['Stadiums_Visited'] = [len(home_mlb_teams_seen_list),len(home_nba_teams_seen_list),len(home_nfl_teams_seen_list),len(home_nhl_teams_seen_list),len(home_mls_teams_seen_list)]
games_dict['%_Visited'] = [mlb_stadiums_visited, nba_stadiums_visited, nfl_stadiums_visited, nhl_stadiums_visited, mls_stadiums_visited]
games_dict['Teams_Seen'] = [len(all_mlb_teams_seen_list),len(all_nba_teams_seen_list),len(all_nfl_teams_seen_list),len(all_nhl_teams_seen_list),len(all_mls_teams_seen_list)]
games_dict['%_Teams_Seen'] = [mlb_teams_seen,nba_teams_seen,nfl_teams_seen,nhl_teams_seen,mls_teams_seen]


games_breakdown_df = pd.DataFrame(games_dict)
games_breakdown_df.loc[:, "%_Visited"] = games_breakdown_df["%_Visited"].map('{:.1%}'.format)
games_breakdown_df.loc[:, "%_Teams_Seen"] = games_breakdown_df["%_Teams_Seen"].map('{:.1%}'.format)
games_breakdown_df.set_index('League')


,League_Teams_Total,Stadiums_Visited,%_Visited,Teams_Seen,%_Teams_Seen
League,,,,,
MLB,32,27,84.4%,32,97.0%
NBA,30,9,30.0%,22,73.3%
NFL,32,1,3.1%,2,6.2%
NHL,32,2,6.2%,5,15.6%
MLS,28,1,3.6%,2,7.1%
